<a href="https://colab.research.google.com/github/DojunPark/Machine_Translation/blob/master/07_transformer_training_trial(ko-en%2C30k).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Oct 23 01:21:33 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8    10W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Preprocessing the training data

- konlpy 라이브러리의 한국어 형태소분석기 Mecab 설치

In [2]:
!pip install konlpy
!sudo apt-get install curl git
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

     |████████████████████████████████| 19.4MB 60.9MB/s 
     |████████████████████████████████| 3.8MB 50.8MB/s 
     |████████████████████████████████| 92kB 13.8MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Reading package lists... Done
Building dependency tree       
Reading state information... Done
curl is already the newest version (7.58.0-2ubuntu3.10).
git is already the newest version (1:2.17.1-1ubuntu0.7).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
Installing automake (A dependency for mecab-ko)
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRe

- 카카오 형태소 분석기(khaiii) 설치

In [5]:
!git clone https://github.com/kakao/khaiii.git
!pip install cmake
!mkdir build
!cd build && cmake /content/khaiii
!cd /content/build/ && make all
!cd /content/build/ && make resource
!cd /content/build && make install
!cd /content/build && make package_python
!pip install /content/build/package_python

Cloning into 'khaiii'...
remote: Enumerating objects: 80, done.
remote: Counting objects: 100% (80/80), done.
remote: Compressing objects: 100% (59/59), done.
remote: Total 957 (delta 25), reused 49 (delta 19), pack-reused 877
Receiving objects: 100% (957/957), 33.03 MiB | 32.12 MiB/s, done.
Resolving deltas: 100% (383/383), done.
-- [hunter] Initializing Hunter workspace (70287b1ffa810ee4e952052a9adff9b4856d0d54)
-- [hunter]   https://github.com/ruslo/hunter/archive/v0.23.34.tar.gz
-- [hunter]   -> /root/.hunter/_Base/Download/Hunter/0.23.34/70287b1
-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-

- spacy 라이브러리의 영어 형태소분석기 설치

In [6]:
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [23]:
from konlpy.tag import Mecab, Komoran
from khaiii import KhaiiiApi
import spacy

khaiii = KhaiiiApi()
mecab = Mecab()
komoran = Komoran()
spacy_en = spacy.load('en')

def tokenize_khaiii(sent):
    list = []
    for word in khaiii.analyze(sent):
        for x in word.morphs:
            list.append(x.lex)
    return list

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

print('tokenization test with sample texts')
print('tokenizing for Korean with khaiii: ', tokenize_khaiii('안녕하세요 저는 건국대학교에 재학 중인 박도준입니다.'))
print('tokenizing for Korean with mecab: ', mecab.morphs('안녕하세요 저는 건국대학교에 재학 중인 박도준입니다.'))
print('tokenizing for Korean with komoran: ', komoran.morphs('안녕하세요 저는 건국대학교에 재학 중인 박도준입니다.'))
print('tokenizing for English: ', tokenize_en('Hello, I am Dojun Park, a student at Konkuk University.'))

tokenization test with sample texts
tokenizing for Korean with khaiii:  ['안녕', '하', '세요', '저', '는', '건국', '대학교', '에', '재학', '중', '이', 'ㄴ', '박도준', '이', 'ㅂ니다', '.']
tokenizing for Korean with mecab:  ['안녕', '하', '세요', '저', '는', '건국대', '학교', '에', '재학', '중', '인', '박도준', '입니다', '.']
tokenizing for Korean with komoran:  ['안녕하세요', '저', '는', '건국대학교', '에', '재학', '중', '이', 'ㄴ', '박', '도준', '이', 'ㅂ니다', '.']
tokenizing for English:  ['Hello', ',', 'I', 'am', 'Dojun', 'Park', ',', 'a', 'student', 'at', 'Konkuk', 'University', '.']


In [9]:
!pip freeze

absl-py==0.10.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
argon2-cffi==20.1.0
asgiref==3.2.10
astor==0.8.1
astropy==4.0.1.post1
astunparse==1.6.3
async-generator==1.10
atari-py==0.2.6
atomicwrites==1.4.0
attrs==20.2.0
audioread==2.1.8
autograd==1.3
Babel==2.8.0
backcall==0.2.0
beautifulsoup4==4.6.0
bleach==3.2.1
blis==0.4.1
bokeh==2.1.1
Bottleneck==1.3.2
branca==0.4.1
bs4==0.0.1
CacheControl==0.12.6
cachetools==4.1.1
catalogue==1.0.0
certifi==2020.6.20
cffi==1.14.3
chainer==7.4.0
chardet==3.0.4
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorama==0.4.4
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.2.2
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cupy-cuda101==7.4.0
cvxopt==1.2.5
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.3
Cython==0.29.21
daft==0.0.4
dask==2.12.0
dataclasses==0.7
datascience==0.10.6
debugpy==1.0.0rc2
decorator==4.4.2
defusedxml==0.6.0
descartes==1.1.0
dill==0.3.2
distributed==1.25.3
Django==3.1.1
dl

In [10]:
!pip uninstall torchtext -y 

Uninstalling torchtext-0.3.1:
  Successfully uninstalled torchtext-0.3.1


In [11]:
!pip install torchtext

     |████████████████████████████████| 4.5MB 11.0MB/s 
     |████████████████████████████████| 1.1MB 43.5MB/s 


# Reducing dataset (30k)

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/news800k.csv')

In [ ]:
df.shape

(801387, 2)

In [ ]:
df_tmp = df[:30000]
df_tmp.sample(10)

,kor,eng
26952,마음 속에 간직할 수 있는 즐거운 경험과 함께하는 소비에서 더 행복을 느끼는 게 요...,It is the younger generation these days that f...
21315,"사실 슬픈 추억 같은 것을 나눈 사이가 전혀 아니었는데, 갑자기 모든 기억이 슬픈 ...","In fact, we never shared anything like sad mem..."
8422,지난 10일 중국 산시성 타이위안에서 귀경객들이 열차에 오르고 있다.,"On the 10th, returnees are getting on the trai..."
28041,오곡이 익어가는 계절 한가운데에 자리잡은 한가위는 예로부터 풍요의 상징이었다.,"Hangawi, in the middle of the season when crop..."
14519,비가 오락가락하고 날도 후덥지근한 장마가 지나면 본격적인 무더위가 시작된다.,When the rain comes and goes and the rainy day...
9849,소실 우려가 있는 공동체의 자료를 보존하는 아카이브 기능을 강화한다.,It enhances the archive's ability to preserve ...
26213,다른 종목에서 부진에 부진을 거듭하는 상황에서 카바디가 성과를 올리며 더 크게 주목...,At the time of being sluggish in other sports ...
10481,연금 수령보다는 세액 공제 목적의 가입자가 더 많다는 의미다.,This means more members have joined for the pu...
25971,특히나 라돈 방사선에 대한 관리·규제가 여러 부처에 흩어져 있다는 점이 이번 사태를...,"In particular, it was pointed out that the man..."
20891,근현대적인 유럽의 건축 양식과 아시아 전통 건축 특징이 조합된 인테리어로 꾸며져 투...,The interior combines modern European architec...


In [ ]:
df_tmp.shape

(30000, 2)

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df_tmp, test_size=0.2, random_state=1)
train_data, valid_data = train_test_split(train_data, test_size=0.25, random_state=1)

In [ ]:
print(train_data.shape)
print(test_data.shape)
print(valid_data.shape)

(18000, 2)
(6000, 2)
(6000, 2)


In [ ]:
train_data.to_csv('/content/drive/My Drive/Colab Notebooks/train2.csv')
test_data.to_csv('/content/drive/My Drive/Colab Notebooks/test2.csv')
valid_data.to_csv('/content/drive/My Drive/Colab Notebooks/valid2.csv')

- tokenizing using torchtext (mecab)

In [27]:
from torchtext.data import Field, TabularDataset, BucketIterator

korean = Field(tokenize=mecab.morphs, lower=True, init_token='<sos>', eos_token='<eos>')
english = Field(tokenize=tokenize_en, lower=True, init_token='<sos>', eos_token='<eos>')

fields = {'kor': ('src', korean), 'eng':('trg', english)}

train_data, valid_data, test_data = TabularDataset.splits(
                                                    path = '/content/drive/My Drive/Colab Notebooks',
                                                    train = 'train2.csv',
                                                    validation = 'valid2.csv',
                                                    test = 'test2.csv',
                                                    format = 'csv',
                                                    fields = fields)

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes f

- tokenizing using torchtext (komoran)

In [36]:
from torchtext.data import Field, TabularDataset, BucketIterator

korean = Field(tokenize=komoran.morphs, lower=True, init_token='<sos>', eos_token='<eos>')
english = Field(tokenize=tokenize_en, lower=True, init_token='<sos>', eos_token='<eos>')

fields = {'kor': ('src', korean), 'eng':('trg', english)}

train_data, valid_data, test_data = TabularDataset.splits(
                                                    path = '/content/drive/My Drive/Colab Notebooks',
                                                    train = 'train2.csv',
                                                    validation = 'valid2.csv',
                                                    test = 'test2.csv',
                                                    format = 'csv',
                                                    fields = fields)

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes f

- tokenizing using torchtext (khaiii)

In [26]:
from torchtext.data import Field, TabularDataset, BucketIterator

korean = Field(tokenize=tokenize_khaiii, lower=True, init_token='<sos>', eos_token='<eos>')
english = Field(tokenize=tokenize_en, lower=True, init_token='<sos>', eos_token='<eos>')

fields = {'kor': ('src', korean), 'eng':('trg', english)}

train_data, valid_data, test_data = TabularDataset.splits(
                                                    path = '/content/drive/My Drive/Colab Notebooks',
                                                    train = 'train2.csv',
                                                    validation = 'valid2.csv',
                                                    test = 'test2.csv',
                                                    format = 'csv',
                                                    fields = fields)

/usr/local/lib/python3.6/dist-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:68: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.', UserWarning)
/usr/local/lib/python3.6/dist-packages/torchtext/data/example.py:52: UserWarning: Example class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes f

In [28]:
print(train_data[0].__dict__)

{'src': ['영화', '가', '관객', '의', '압도', '적', '인', '지지', '를', '받', '으며', '덩달', '아', '퀸', '신드롬', '까지', '일으켰', '다', '.'], 'trg': ['the', 'movie', 'even', 'created', 'queen', 'syndrome', 'with', 'overwhelming', 'support', 'from', 'the', 'audience', '.']}


- make vocab list

In [37]:
korean.build_vocab(train_data, min_freq=2)
english.build_vocab(train_data, min_freq=2)

# Model designing

In [38]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter


class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        dropout,
        max_length,
        device):

        super(Transformer, self).__init__()
        self.src_word_embedding = nn.Embedding(src_vocab_size, embedding_size)
        self.src_position_embedding = nn.Embedding(max_len, embedding_size)
        self.trg_word_embedding = nn.Embedding(trg_vocab_size, embedding_size)
        self.trg_position_embedding = nn.Embedding(max_len, embedding_size)
        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout)
        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        # src shape: (src_len, N)
        src_mask = src.transpose(0, 1) == self.src_pad_idx
        # (N, src_len)
        return src_mask

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_positions = (
            torch.arange(0, src_seq_length).unsqueeze(1).expand(src_seq_length, N)
            .to(self.device)
        )

        trg_positions = (
            torch.arange(0, trg_seq_length).unsqueeze(1).expand(trg_seq_length, N)
            .to(self.device)
        )

        embed_src = self.dropout(
            (self.src_word_embedding(src) + self.src_position_embedding(src_positions))
        )

        embed_trg = self.dropout(
            (self.trg_word_embedding(trg) + self.trg_position_embedding(trg_positions))
        )

        src_padding_mask = self.make_src_mask(src)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(
            self.device)

        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask = src_padding_mask,
            tgt_mask = trg_mask
        )
        out = self.fc_out(out)

        return out

In [39]:
# Setup the training phase
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
load_model = False
save_model = True

# Training hyperparameters
num_epochs = 5
learning_rate = 1e-4
batch_size = 32

# Model hyperparameters
src_vocab_size = len(korean.vocab)
trg_vocab_size = len(english.vocab)
embedding_size = 512
num_heads = 8
num_encoder_layers = 6  # in the paper 6
num_decoder_layers = 6
dropout = 0.10
max_len = 100
forward_expansion = 4
src_pad_idx = english.vocab.stoi['<pad>']

# Tensorboard for nice plots
writer = SummaryWriter('runs/loss_plot')
step = 0

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = batch_size,
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device
)

model = Transformer(
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_len,
    device).to(device)

optimizer = optim.Adam(model.parameters(), lr=learning_rate)
pad_idx = english.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx)

/usr/local/lib/python3.6/dist-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


# Define functions for test

In [40]:
import torch
import spacy
from torchtext.data.metrics import bleu_score
import sys


def translate_sentence(model, sentence, korean, english, device, max_length=50):
    
    # tokens = mecab.morphs(sentence)
    tokens = komoran.morphs(sentence)
    # tokens = tokenize_khaiii(sentence)

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, korean.init_token)
    tokens.append(korean.eos_token)

    # Go through each korean token and convert to an index
    text_to_indices = [korean.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    # remove start token
    return translated_sentence[1:]


def bleu(data, model, korean, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]

        prediction = translate_sentence(model, src, korean, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)

    return bleu_score(outputs, targets)


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

# Training the model (tokenizer = mecab)

In [35]:
sentence = '이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다.'  # df['kor'][117]

for epoch in range(num_epochs):

    model.eval()
    model.train()

    for batch_idx, batch in enumerate(train_iterator):
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)

        # forward prop
        output = model(inp_data, target[:-1])
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)
        optimizer.zero_grad()

        loss = criterion(output, target)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()
        
        writer.add_scalar('Training loss', loss, global_step=step)
        step += 1


    translated_sentence = translate_sentence(model, sentence, korean, english, device, max_length = 100)
    translated_sentence = ' '.join(translated_sentence)
    translated_sentence = translated_sentence.replace(' ,', ',')
    translated_sentence = translated_sentence.replace(' .', '.')
    translated_sentence = translated_sentence.replace(' <eos>', '')

    print(f'[Epoch] {epoch+1} / {num_epochs}')
    print(f'[Loss] {loss:.4f}')
    print(f'[Exsample] {sentence} >>> {translated_sentence}')
    print(f'[BLEU score] {bleu(test_data, model, korean, english, device):.4f}')
    print('[Training time] {:.2f} min.'.format((time.time() - start) / 60))

/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


[Epoch] 1 / 5
[Loss] 5.7114
[Exsample] 이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다. >>> there are a <unk> of the time of the <unk>, which are not a <unk>.


NotImplementedError: ignored

# Training the model (tokenizer = komoran)

In [41]:
sentence = '이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다.'  # df['kor'][117]

for epoch in range(num_epochs):

    model.eval()
    model.train()

    for batch_idx, batch in enumerate(train_iterator):
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)

        # forward prop
        output = model(inp_data, target[:-1])
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)
        optimizer.zero_grad()

        loss = criterion(output, target)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()
        
        writer.add_scalar('Training loss', loss, global_step=step)
        step += 1


    translated_sentence = translate_sentence(model, sentence, korean, english, device, max_length = 100)
    translated_sentence = ' '.join(translated_sentence)
    translated_sentence = translated_sentence.replace(' ,', ',')
    translated_sentence = translated_sentence.replace(' .', '.')
    translated_sentence = translated_sentence.replace(' <eos>', '')

    print(f'[Epoch] {epoch+1} / {num_epochs}')
    print(f'[Loss] {loss:.4f}')
    print(f'[Exsample] {sentence} >>> {translated_sentence}')
    print(f'[BLEU score] {bleu(test_data, model, korean, english, device):.4f}')
    print('[Training time] {:.2f} min.'.format((time.time() - start) / 60))

/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


[Epoch] 1 / 5
[Loss] 5.7490
[Exsample] 이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다. >>> the <unk> is the world of the <unk> of the <unk> of the market of the <unk>.


AttributeError: ignored

# Training the model (tokenizer = khaiii)

In [19]:
sentence = '이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다.'  # df['kor'][117]

for epoch in range(num_epochs):

    model.eval()
    model.train()

    for batch_idx, batch in enumerate(train_iterator):
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)

        # forward prop
        output = model(inp_data, target[:-1])
        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)
        optimizer.zero_grad()

        loss = criterion(output, target)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

        optimizer.step()
        
        writer.add_scalar('Training loss', loss, global_step=step)
        step += 1


    translated_sentence = translate_sentence(model, sentence, korean, english, device, max_length = 100)
    translated_sentence = ' '.join(translated_sentence)
    translated_sentence = translated_sentence.replace(' ,', ',')
    translated_sentence = translated_sentence.replace(' .', '.')
    translated_sentence = translated_sentence.replace(' <eos>', '')

    print(f'[Epoch] {epoch+1} / {num_epochs}')
    print(f'[Loss] {loss:.4f}')
    print(f'[Exsample] {sentence} >>> {translated_sentence}')
    print(f'[BLEU score] {bleu(test_data, model, korean, english, device):.4f}')
    print('[Training time] {:.2f} min.'.format((time.time() - start) / 60))

/usr/local/lib/python3.6/dist-packages/torchtext/data/batch.py:23: UserWarning: Batch class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)


[Epoch] 1 / 5
[Loss] 5.7290
[Exsample] 이 지역의 많은 공장들이 다른 곳에 외주를 주고 있다. >>> the first is the first of the first of the first of the first of the <unk>.


AttributeError: ignored

# Conclusion

**macab() 형태소분석기를 사용하였을 경우 bleu()함수에서 다음과 같은 에러가 발생**

NotImplementedError: Wrong number or type of arguments for overloaded function 'Tagger_parse'. <br/>
  Possible C/C++ prototypes are: <br/>
    MeCab::Tagger::parse(MeCab::Model const &,MeCab::Lattice *)<br/>
    MeCab::Tagger::parse(MeCab::Lattice *) const<br/>
    MeCab::Tagger::parse(char const *)<br/><br/>


**komoran() 형태소분석기를 사용하였을 경우 bleu()함수에서 다음과 같은 에러가 발생**

AttributeError: 'list' object has no attribute 'split' <br/><br/>


**khaiii() 형태소분석기를 사용하였을 경우 bleu()함수에서 다음과 같은 에러가 발생**

AttributeError: 'list' object has no attribute 'encode' <br/><br/>



=>> 형태소분석기의 문제가 아닌, bleu()함수의 문제일 수 있으니, bleu()함수의 내부 구조를 확인해보고, bleu score를 측정할 수 있는 다른 방법도 찾아보아야 할 것
